# Imdb sentiment analysis dataset

https://keras.io/datasets/

http://www.samyzaf.com/ML/imdb/imdb.html

In [79]:
import keras
from keras.datasets import imdb
from keras import models
from keras import layers

import pandas as pd
import numpy as np
import re
import math
import nltk

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten

from sklearn.feature_extraction.text import TfidfVectorizer

In [80]:
#On garde seulement les 12000 mots les plus frequents dans les données d'entraînement
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=12000)
print(len(X_train), 'train')
print(len(X_test), 'test')
print(X_train[0])

25000 train
25000 test
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


## Pré-traitement des données

In [81]:
### Décodage du text
word_index = imdb.get_word_index() #téléchargement du dico mot-chiffre
#print(word_index)
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) #traduction du dico en chiffre-mot
#print(sorted( reverse_word_index.items(), key=lambda t: t[1]))

decoded_review = ' '.join([reverse_word_index.get(i - 3, '') for i in X_train[0]]) #le mot le plus frequent commence a l'indice numero 3
#print(decoded_review)

In [78]:
"""
wordDict = word_index.copy()

for i in wordDict.keys():
    wordDict[i] = 0
"""
"""
split_1 = re.split("\W+", decoded_review)
print(split_1)
print()
wordDict = dict.fromkeys(set(split_1), 0)
print(wordDict)
print()
for w in split_1:
    wordDict[w]+=1
    
listSplit = []
for i in range(0, len(X_train)):
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '') for i in X_train[i]])
    split = re.split("\W+", decoded_review)
    listSplit.append(split)
    wordDict.update(dict.fromkeys(set(split), 0))
    
print(listSplit)
"""
# print(wordDict)

'\nsplit_1 = re.split("\\W+", decoded_review)\nprint(split_1)\nprint()\nwordDict = dict.fromkeys(set(split_1), 0)\nprint(wordDict)\nprint()\nfor w in split_1:\n    wordDict[w]+=1\n'

In [82]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

In [83]:
x_train = vectorizer.fit_transform(X_train)


AttributeError: 'list' object has no attribute 'lower'

In [47]:
# Our vectorized labels
print(Y_train[0])
y_train = np.asarray(Y_train).astype('float32')
print(y_train[0])

y_test = np.asarray(Y_test).astype('float32')

1
1.0


In [45]:
def vectorize_sequences(sequences, dimension=12500):
    #Creer une matrice de zero de taille len(sequences)X dimension
    results = np.zeros((len(sequences), dimension))
    
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results


x_train = vectorize_sequences(X_train)
print(x_train[0])

x_test = vectorize_sequences(X_test)

[0. 1. 1. ... 0. 0. 0.]


## Model Simple

In [44]:
model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=(12500,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               6400512   
_________________________________________________________________
dense_7 (Dense)              (None, 16)                8208      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 6,408,737
Trainable params: 6,408,737
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [60]:
model.fit(x_train, y_train, 
              epochs = 6,
              validation_data=(x_test, y_test))

ValueError: You are passing a target array of shape (25000, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [ ]:
results = mon_model.evaluate(x_mnist_test, y_mnist_test) 
print("Evaluate ", results)